In [10]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Cargar el modelo entrenado
model_path = "svm_asl_model.pkl"  # Cambia a la ruta de tu modelo
model = joblib.load(model_path)

# Configuración de Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Iniciar captura de video
cap = cv2.VideoCapture(0)

# Función para extraer landmarks
def extract_landmarks(image):
    """Extrae landmarks de la imagen procesada por Mediapipe."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Extraer los 21 landmarks de la mano
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])  # Coordenadas normalizadas
            # Aplanar el vector
            return np.array(landmarks).flatten()
    return None

# Bucle principal de predicción
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("No se pudo capturar la imagen de la cámara.")
        break

    # Extraer landmarks de la imagen capturada
    landmarks = extract_landmarks(frame)

    if landmarks is not None:
        # Redimensionar los landmarks para ajustarlos al modelo
        landmarks = landmarks.reshape(1, -1)
        
        # Predecir la letra con el modelo cargado
        predicted_letter = model.predict(landmarks)[0]
        
        # Dibujar la letra predicha en la imagen
        cv2.putText(frame, f"Letra: {predicted_letter}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        # Informar si no se detectan landmarks
        cv2.putText(frame, "Mano no detectada", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (0, 0, 255), 2, cv2.LINE_AA)

    # Mostrar el video
    cv2.imshow("ASL Recognition", frame)

    # Salir si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()
hands.close()


In [11]:
import numpy as np

def extract_landmarks(results):
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        # Extraer coordenadas x, y, z de los 21 landmarks
        landmarks = []
        for lm in hand_landmarks.landmark:
            landmarks.append([lm.x, lm.y, lm.z])
        return np.array(landmarks).flatten()  # Aplanar el array
    return np.zeros(63)  # Devuelve un vector nulo si no hay mano
